### Adaptive time steps (variable time resolution) for reaction `A <-> B`,
with 1st-order kinetics in both directions, taken to equilibrium

Same as the experiment _"react_1"_ , but with adaptive variable time steps

LAST REVISED: May 16, 2023

In [1]:
import set_path      # Importing this module will add the project's home directory +to sys.path

Added 'D:\Docs\- MY CODE\BioSimulations\life123-Win7' to sys.path


In [2]:
from experiments.get_notebook_info import get_notebook_basename

from src.modules.reactions.reaction_data import ReactionData as chem
from src.modules.reactions.reaction_dynamics import ReactionDynamics

import numpy as np
import pandas as pd
import plotly.express as px
from src.modules.visualization.graphic_log import GraphicLog

In [3]:
# Initialize the HTML logging (for the graphics)
log_file = get_notebook_basename() + ".log.htm"    # Use the notebook base filename for the log file

# Set up the use of some specified graphic (Vue) components
GraphicLog.config(filename=log_file,
                  components=["vue_cytoscape_1"],
                  extra_js="https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.21.2/cytoscape.umd.js")

-> Output will be LOGGED into the file 'react_2.log.htm'


# Initialize the System
Specify the chemicals and the reactions

In [4]:
# Specify the chemicals
chem_data = chem(names=["A", "B"])

# Reaction A <-> B , with 1st-order kinetics in both directions
chem_data.add_reaction(reactants=["A"], products=["B"], 
                       forward_rate=3., reverse_rate=2.)

In [5]:
chem_data.describe_reactions()

Number of reactions: 1 (at temp. 25 C)
0: A <-> B  (kF = 3 / kR = 2 / Delta_G = -1,005.13 / K = 1.5) | 1st order in all reactants & products


In [6]:
# Send a plot of the network of reactions to the HTML log file
graph_data = chem_data.prepare_graph_network()
GraphicLog.export_plot(graph_data, "vue_cytoscape_1")

[GRAPHIC ELEMENT SENT TO LOG FILE `react_2.log.htm`]


# Start the simulation

In [7]:
dynamics = ReactionDynamics(reaction_data=chem_data)

In [8]:
# Initial concentrations of all the chemicals, in index order
dynamics.set_conc([10., 50.])

dynamics.describe_state()

SYSTEM STATE at Time t = 0:
2 species:
  Species 0 (A). Conc: 10.0
  Species 1 (B). Conc: 50.0


In [9]:
dynamics.get_history()

,SYSTEM TIME,A,B,caption
0,0.0,10.0,50.0,Initial state


## Run the reaction

In [10]:
dynamics.set_diagnostics()       # To save diagnostic information about the call to single_compartment_react()

# All of these are the currently the default values, but subject to change
dynamics.set_thresholds(thresholds={"low": 0.5, "high": 0.8, "abort": 1.44, "reduction_factor": 2.})

dynamics.single_compartment_react(initial_step=0.1, target_end_time=1.2,
                                  variable_steps=True, 
                                  snapshots={"initial_caption": "1st reaction step",
                                             "final_caption": "last reaction step"}
                                  )

INFO: the tentative time step (0.1) leads to a least one norm value > its ABORT threshold:
      -> will backtrack, and re-do step with a SMALLER delta time of 0.05 [Step started at t=0, and will rewind there]
INFO: the tentative time step (0.05) leads to a least one norm value > its ABORT threshold:
      -> will backtrack, and re-do step with a SMALLER delta time of 0.025 [Step started at t=0, and will rewind there]
INFO: the tentative time step (0.025) leads to a least one norm value > its ABORT threshold:
      -> will backtrack, and re-do step with a SMALLER delta time of 0.0125 [Step started at t=0, and will rewind there]
Some steps were backtracked and re-done, to prevent negative concentrations or excessively large concentration changes
17 total step(s) taken


## The flag _variable_steps_ automatically adjusts up or down the time step,  whenever the changes of concentrations are, respectively, "slow" or "fast" (as determined using the specified _thresholds_ )

In [11]:
df = dynamics.get_history()   # The system's history, saved during the run of single_compartment_react()
df

,SYSTEM TIME,A,B,caption
0,0.000000,10.000000,50.000000,Initial state
1,0.012500,10.875000,49.125000,1st reaction step
2,0.025000,11.695312,48.304688,
3,0.043750,12.848877,47.151123,
4,0.062500,13.894295,46.105705,
5,0.081250,14.841705,45.158295,
6,0.109375,16.129590,43.870410,
7,0.123438,16.682978,43.317022,
8,0.144531,17.454695,42.545305,
9,0.176172,18.490183,41.509817,


In [12]:
dynamics.explain_time_advance()

From time 0 to 0.025, in 2 steps of 0.0125
From time 0.025 to 0.08125, in 3 steps of 0.0188
From time 0.08125 to 0.1094, in 1 step of 0.0281
From time 0.1094 to 0.1234, in 1 step of 0.0141
From time 0.1234 to 0.1445, in 1 step of 0.0211
From time 0.1445 to 0.2078, in 2 steps of 0.0316
From time 0.2078 to 0.3027, in 2 steps of 0.0475
From time 0.3027 to 0.3739, in 1 step of 0.0712
From time 0.3739 to 0.4807, in 1 step of 0.107
From time 0.4807 to 0.6409, in 1 step of 0.16
From time 0.6409 to 0.8812, in 1 step of 0.24
From time 0.8812 to 1.242, in 1 step of 0.36


## Notice how the reaction proceeds in smaller steps in the early times, when [A] and [B] are changing much more rapidly
### That resulted from passing the flag _variable_steps=True_ to single_compartment_react()

## Scrutinizing some instances of step-size changes

### Detailed Example 1: **going from 0.1375 to 0.1875**    

In [13]:
lookup = dynamics.get_history(t_start=0.1375, t_end=0.1875)
lookup

,SYSTEM TIME,A,B,caption
8,0.144531,17.454695,42.545305,
9,0.176172,18.490183,41.509817,


In [14]:
delta_concentrations = dynamics.extract_delta_concentrations(lookup, 7, 8, ['A', 'B'])
delta_concentrations

KeyError: 7

As expected by the 1:1 stoichiometry, delta_A = - delta_B

The above values coud also be looked up from the diagnostic data, since we only have 1 reaction:

In [ ]:
rxn_data = dynamics.get_diagnostic_rxn_data(rxn_index=0)

In [ ]:
rxn_data[0:12]

In [ ]:
delta_row = dynamics.get_diagnostic_rxn_data(rxn_index=0, t=0.1375) # Locate the row with the interval's start time
delta_row

In [ ]:
delta_row[["Delta A", "Delta B"]].to_numpy()   # Gives same value as delta_concentrations, above

In [ ]:
adjusted_L2_rate = dynamics.norm_A(delta_concentrations)  # A measure of how large delta_concentrations is
adjusted_L2_rate

In [ ]:
dynamics.adjust_speed(delta_concentrations)

#### The above conclusion is that the step will be **HALVED** at the next round : that's because the adjusted_L2_rate > the "high" value given in the argument _thresholds={"low": 0.5, "high": 0.8, "abort": 1.44, "reduction_factor": 2.}_ , and the and the step_determiner() function returned 0.5

### Detailed Example 2: **going from 0.1875 to 0.2125**   

In [ ]:
lookup = dynamics.get_history(t_start=0.1875, t_end=0.2125)
lookup

In [ ]:
delta_concentrations = dynamics.extract_delta_concentrations(lookup, 8, 9, ['A', 'B'])
delta_concentrations

Note how substantially smaller _delta_concentrations_ is, compared to the previous example

In [ ]:
adjusted_L2_rate = dynamics.norm_A(delta_concentrations)  # A measure of how large delta_concentrations is
adjusted_L2_rate

In [ ]:
dynamics.adjust_speed(delta_concentrations)

#### The above conclusion is that the step will be **DOUBLED** at the next round : that's because the adjusted_L2_rate < the "low" value given in the argument _thresholds={"low": 0.5}_ , and the step_determiner() function returned 2

# Check the final equilibrium

In [ ]:
# Verify that the reaction has reached equilibrium
dynamics.is_in_equilibrium()

# Plots of changes of concentration with time

In [ ]:
dynamics.plot_curves(colors=['blue', 'orange'])

## Note how the left-hand side of this plot is much smoother than it was in experiment `react_1`, where no adaptive time steps were used!

In [ ]:
dynamics.plot_curves(colors=['blue', 'orange'], show_intervals=True)

#### Compare the above with the fixed step sizes (always 0.1) of experiment `react_1`

In [ ]:
dynamics.plot_step_sizes(show_intervals=True)

# Diagnostics of the run may be investigated as follows:  
_(note - this is possible because we make a call to set_diagnostics() prior to running the simulation)_

In [ ]:
dynamics.get_diagnostic_conc_data()   # This will be complete, even if we only saved part of the history during the run

In [ ]:
dynamics.get_diagnostic_rxn_data(rxn_index=0)      # For the 0-th reaction (the only reaction in our case)

### Note that diagnostic data with the DELTA Concentrations - above and below - also record the values that were considered (but not actually used) during ABORTED steps

In [ ]:
dynamics.get_diagnostic_decisions_data()